In [18]:
import pandas as pd
import backtrader as bt
from datetime import datetime
import yfinance as yf
from pandas_datareader import data as pdr

In [19]:
# 通过yfinance下载
yf.pdr_override()  # <== that's all it takes :-)
symbol = "AAPL"
df = pdr.get_data_yahoo(symbol, '2020-01-01','2022-12-31')# Set my column names
df.columns=['high', 'low', 'open', 'close', 'volume', 'adj_close']
data=bt.feeds.PandasData(dataname=df)


[*********************100%***********************]  1 of 1 completed


In [20]:
class Strategy1(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s: %s' % (dt.isoformat(), txt))

    def __init__(self) -> None:
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.sma5 = bt.ind.SMA(self.datas[0], period=5)
        self.rsi = bt.ind.RelativeStrengthIndex()

    def next(self):
        if self.order:
            return
        if self.dataclose > self.sma5:
            self.log('buy')
        if self.rsi > self.dataclose:
            self.log('sell')


In [ ]:
cerebro=bt.Cerebro()
cerebro.addstrategy(Strategy1)
cerebro.adddata(data)
cerebro.run()
cerebro.plot()